# Jupyter Notebook to Convert the Spreadsheet CSV in Mapping dictionaries

In [1]:
import numpy as np
import pandas as pd
import os
import sys

AttributeError: module 'numpy' has no attribute '__version__'

In [ ]:
# Change the working directory to the script's directory

working_dir = "/Users/peterkinget/iCloudDrive/Work/MOSBIUS/MOSbiusTools/MOSbiusCADFlow/MOSbiusV2Tools/MOSbiusV2Tools/tmp/csv_to_dict"
os.chdir(working_dir)

# Define the file path
filename = "MOSbiusV2_pin_map - Pin List_20250501.csv"

# Check if the script is being run from the correct directory
if not os.path.exists(filename):
    print(f"Error: The script must be run from the directory containing {filename}.")
    sys.exit(1) 

In [ ]:
# Read the CSV file
df_orig = pd.read_csv(filename, skiprows=13)  # Skip the first 10 rows of metadata

In [ ]:
columns = df_orig.columns.tolist()   # Print the column names
columns

In [ ]:
save_columns = ['Package Pin no.', 'Switch Matrix Pin No. ', 'Pin Name', ]
# Filter the DataFrame to include only the specified columns
df = df_orig[save_columns]
# Rename the columns
df.rename(columns={
    'Package Pin no.': 'pin_number',
    'Switch Matrix Pin No. ': 'sw_matrix_pin_number',
    'Pin Name': 'pin_name'
}, inplace=True)    
# Remove the empty rows
df = df.dropna(how='all')

In [ ]:
print(df)
print(df.columns)

In [ ]:
# Make a dictionary from the DataFrame that maps Pin Name to Pin No.
pin_map = {}
for index, row in df.iterrows():
    pin_name = row['pin_name']
    pin_number = row['pin_number']
    if pd.notna(pin_name) and pd.notna(pin_number):
        pin_map[pin_name] = pin_number
pin_name_to_number = pin_map

# Make a dictionary from the Dataframe that maps Pin numbers to pin names
pin_number_to_name = {}
for index, row in df.iterrows():
    pin_number = row['pin_number']
    pin_name = row['pin_name']
    if pd.notna(pin_number) and pd.notna(pin_name):
        pin_number_to_name[pin_number] = pin_name

# Make a dictionary from the Dataframe that maps the pin name to the switch matrix pin number
sw_matrix_pin_map = {}
for index, row in df.iterrows():
    pin_name = row['pin_name']
    sw_matrix_pin_number = row['sw_matrix_pin_number']
    if pd.notna(pin_name) and pd.notna(sw_matrix_pin_number):
        sw_matrix_pin_map[pin_name] = sw_matrix_pin_number

pin_name_to_sw_matrix_pin_number = sw_matrix_pin_map

In [ ]:
# Save the dictionaries to JSON files
import json 
with open('pin_name_to_number.json', 'w') as f:
    json.dump(pin_name_to_number, f, indent=4)
with open('pin_number_to_name.json', 'w') as f:
    json.dump(pin_number_to_name, f, indent=4)  
with open('pin_name_to_sw_matrix_pin_number.json', 'w') as f:   
    json.dump(pin_name_to_sw_matrix_pin_number, f, indent=4)
# Save the DataFrame to a new CSV file
df.to_csv('MOSbiusV2_pin_map - Pin List_cleaned.csv', index=False)

In [ ]:
pin_name_to_number 

In [ ]:
pin_number_to_name

In [ ]:
pin_name_to_sw_matrix_pin_number

In [ ]:
import pandas as pd

def group_register_bits(csv_file_path):
    """
    Reads a CSV file and groups register bits into sets of 5 bits for each device.

    Args:
        csv_file_path (str): Path to the CSV file.

    Returns:
        dict: A dictionary where keys are device names and values are dictionaries
              mapping bit labels (1, 2, 4, 8, 16) to register numbers.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Initialize a dictionary to store grouped registers
    grouped_registers = {}

    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        reg_name = row['Reg Names']
        reg_number = row['Reg Numbers']

        # Extract the base device name (e.g., "DCC1_P_L" from "DCC1_P_L_1")
        base_name = "_".join(reg_name.split("_")[:-1])

        # Extract the bit label (e.g., "1" from "DCC1_P_L_1")
        bit_label = int(reg_name.split("_")[-1])

        # Add the register to the corresponding device group
        if base_name not in grouped_registers:
            grouped_registers[base_name] = {}
        grouped_registers[base_name][bit_label] = reg_number

    # Ensure each group is sorted by the bit label
    for base_name in grouped_registers:
        grouped_registers[base_name] = dict(sorted(grouped_registers[base_name].items()))

    return grouped_registers

# Example usage
csv_file_path = "MOSbiusV2_register_map_20250503.csv"
grouped_registers = group_register_bits(csv_file_path)

# Print the grouped registers
for device, bits in grouped_registers.items():
    print(f"{device}: {bits}")
    

In [ ]:
# Save the dictionaries to JSON files

with open('device_name_to_sizing_registers.json', 'w') as f:
    json.dump(grouped_registers, f, indent=4)


## Generating Switch Matrix Register Map

In [ ]:
import csv
import json
import os

In [ ]:
# Change the working directory to the script's directory

working_dir = "/Users/peterkinget/iCloudDrive/Work/MOSBIUS/MOSbiusTools/MOSbiusCADFlow/MOSbiusV2Tools/MOSbiusV2Tools/tmp/csv_to_dict"
os.chdir(working_dir)

# Define the file path
file_path = "MOSbiusV2_sw_matrix_register_map_20250503.csv"

# Check if the script is being run from the correct directory
if not os.path.exists(file_path):
    print(f"Error: The script must be run from the directory containing {filename}.")
    sys.exit(1) 

In [ ]:
def parse_csv_to_switch_matrix_register_map(file_path):
    """
    Parses the CSV file into a nested dictionary for easy lookup by pin/internal_pin and bus type.
    
    Args:
        file_path (str): Path to the CSV file.
    
    Returns:
        dict: A nested dictionary where keys are pin/internal_pin names, and values are dictionaries
              mapping bus types (SBUS or RBUS) and columns (a, b, or RBUS index) to register numbers.
    """
    register_map = {}

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)
        headers = next(csv_reader)  # Read the header row
        headers = next(csv_reader)  # Read the header row
        bus_headers = headers[2:-1]  # Extract bus headers (SBUS and RBUS columns)

        for row in csv_reader:
            if not any(row):  # Skip empty rows
                continue

            pin_type = row[0].strip()  # e.g., "swmatrix_pin" or "swmatrix_int_pin"
            pin_name = row[1].strip()  # e.g., "1", "A", etc.

            if pin_type and pin_name:
                pin_key = f"{pin_type}_{pin_name}"
                pin_key = pin_name
                register_map[pin_key] = {}

                for i, bus_header in enumerate(bus_headers):
                    if bus_header:  # Skip empty headers
                        # bus_type, column = bus_header[:-1], bus_header[-1]  # e.g., "SBUS1a" -> "SBUS1", "a"
                        register_value = row[i + 2].strip()  # Offset by 2 to match column index
                        if register_value:  # Only add non-empty register values
                            register_map[pin_key][bus_header] = register_value
    return register_map


In [ ]:
switch_matrix_register_map = parse_csv_to_switch_matrix_register_map(file_path)

# Query example
pin = "A"
bus = "RBUS1"
print(f"Register for {pin} on {bus} is {switch_matrix_register_map[pin][bus]}")

In [ ]:
# Save the dictionaries to JSON files

with open('switch_matrix_register_map.json', 'w') as f:
    json.dump(switch_matrix_register_map, f, indent=4)


In [ ]:
%pwd